# Dataset pre-processing

As data was extracted during the review this was stored in a formatted spreadsheet. Before analysis in Python we clean and pre-process the dataset.  This notebook provides an overview of the code to read in the data extracted from the review.

> **WORK IN PROCESS**: TM has added several queries regarding the review dataset that need to be reviewed before progressing the study.

::: {.callout-caution collapse="true"}
## TM Queries about data

* **TM query (1)**: can we drop column index 27? This is labelled 27 and is all values are null.  I've removed it in the code below...

* **TM query (2)**: there were three blank lines at the end of the CSV. I've removed this in the updated load routine.

* **TM query (3)**: what does DP mean in "used?"

* **TM query (4)**: target authors: is this a manual field you have included? or did you have a formula in Excel? It appears to be different from your calculation at the end?

* **TM query (5)**: To discuss with others -> I have excluded all studies not in English. This is standard practice for a review (basically we would need to translate them if we wanted to include.)
:::

## 1. Imports

In [1]:
import numpy as np
import pandas as pd

from typing import Optional

## 2. Constants

In [2]:
REVIEW_CSV_FILE_PATH = "PY_STRESS.csv"

## 3. Functions to read and clean dataset

We have implemented a pipeline to read and clean of the extracted data using `pandas`

### 3.1 Cleaning helping functions

There are four supporting functions to the main pipeline.

**Three functions format column headers:**
* `recode_whitespace` - replace all spaces in column headers with an underscore "_"
* `strip_punctuation` - remove any English punctuation from column headers e.g. "?" or "'"
* `cols_to_lower` - ensure all column headers are lowercase

The final function `drop_non_english_language` drops all studies that are not reported in the the English language.


In [3]:
def recode_whitespace(df: pd.DataFrame) -> pd.DataFrame:
    '''
    recode whitespace as "_" and strip head/tail whitespace just in case
    '''
    # strip leading and lagging white space
    df.columns = df.columns.str.strip()
    # replace remaining whitespace with "_"
    df.columns = df.columns.str.replace(" ", "_")
    return df

In [4]:
def strip_punctuation(df: pd.DataFrame) -> pd.DataFrame:
    '''
    strip select punction from column headers
    '''
    df.columns = df.columns.str.replace("?", "")
    df.columns = df.columns.str.replace("'", "")
    df.columns = df.columns.str.replace("-", "_")
    df.columns = df.columns.str.replace("/", "")
    return df

In [5]:
def cols_to_lower(df: pd.DataFrame) -> pd.DataFrame:
    """
    Convert all column names in a dataframe to lower case

    Params:
    ------
    df - pandas.DataFrame

    Returns:
    -------
    out: pandas.DataFrame
    """
    new_cols = [c.lower() for c in df.columns]
    df.columns = new_cols
    return df

In [6]:
def drop_non_english_language(df: pd.DataFrame) -> pd.DataFrame:
    '''
    Check for "Not in English" recorded in any column
    Return all other rows.
    '''
    return df[~df.isin(['Not in English']).any(axis=1)]

### 3.2 Main load and clean function

The main function makes use of `pandas`method chaining functions.

In [7]:
#| code-fold: true
def load_review_dataset(
    path: Optional[str] = REVIEW_CSV_FILE_PATH,
) -> pd.DataFrame:
    """Read full data extraction data set for the review from a CSV file.
    Returns cleaned dataset.

    Assumes data is stored in .csv

    Cleaning pipeline for dataset:
    1. drop redunance columns
    2. drop rows that contain all NAs
    3. drop rows that are no in English
    3. rename columns with complex strings
    4. strip all punctuation from column headers
    5. column headers to lower case
    6. replace all whitespace in headers with "_"
    7. convert all blank strings in cells to NaN
    8. recode variables to be internally consistent in naming
    9. perform type conversion for integer fields
    10. type conversion for categorical fields
    

    Parameters:
    ----------
    path: str, optional (default=REVIEW_CSV_FILE_PATH)
        path or URL for review dataset.

    Returns:
    --------
    out: pd.DataFrame

    """

    # SETUP FOR DATASET CLEAN

    # cols to drop from data read in
    cols_to_remove = [
        "Unnamed: 0",
        "...27",
        "Questions to be asked from authors / experts",
        "Fatemeh's Note",
        "Note",
    ]

    # simple type conversions
    type_conversions = {"year": "UInt16"}

    # renaming of columns to names suitable for analysis
    new_labels = {
        "1. Objectives (purpose, model outputs, aims of experimentation)": "stress_objectives",
        "2. Logic (base model overview diagram, base model logic, scenario logic, algorithms, components)": "stress_logic",
        "3. Data (data sources, input parameters, preprocessing, assumptions": "stress_logic",
        "4. Experimentation (initialisation, run length, estimation approach)": "stress_exp",
        "5. Implementation (software and programming language, random sampling, model execution, system specification)": "stress_imp",
        "6. Code access (computer model sharing statement)": "stress_code",
    }

    # used to recode variables so they are consistent.
    recoded_variables = {"used": {"NO": "No"}}

    # DATA CLEANING PIPELINE
    clean = (
        pd.read_csv(path, index_col="No")
        # drop redundant index column
        .drop(labels=cols_to_remove, axis=1)
        # drop all blank rows (this will remove the blank 3 rows at end of CSV)
        .dropna(how="all")
        # drop all studies that do not use an English language
        .pipe(drop_non_english_language)
        # rename verbose column headers
        .rename(columns=new_labels)
        # remove any punctutation i.e. "?" and "-"
        .pipe(strip_punctuation)
        # all columns headers to lower case
        .pipe(cols_to_lower)
        # replace all whitespace with "_" in col headers
        .pipe(recode_whitespace)
        # replace all whitespace and blank strings in fields with NaN
        .replace(r"^\s*$", np.nan, regex=True)
        # recoded variables e.g. used "NO" becomes "No"
        .replace(recoded_variables)
        # update the type of columns to int where needed
        .astype(type_conversions)
        # categorical variables
        .assign(
            used=lambda x: pd.Categorical(x["used"]),
            type_of_paper=lambda x: pd.Categorical(x["type_of_paper"]),
            partially=lambda x: pd.Categorical(x["partially"]),
            method=lambda x: pd.Categorical(x["method"]),
            software=lambda x: pd.Categorical(x["software"]),
            source_code_access=lambda x: pd.Categorical(
                x["source_code_access"]
            ),
            application_area=lambda x: pd.Categorical(x["application_area"]),
            target_authors=lambda x: pd.Categorical(x["target_authors"]),
            stress_implementation=lambda x: pd.Categorical(
                x["stress_implementation"]
            ),
            hybridisation=lambda x: pd.Categorical(x["hybridisation"])
        )
    )
    return clean

## 4. Example read in and clean of full dataset

Here we run the pre-processing of the main review dataset and then examine the DataFrame information and peak at the head and tail.

In [8]:
# read in data.
clean = load_review_dataset()
clean.info()

<class 'pandas.core.frame.DataFrame'>
Index: 165 entries, 1 to 171
Data columns (total 25 columns):
 #   Column                 Non-Null Count  Dtype   
---  ------                 --------------  -----   
 0   publication            165 non-null    object  
 1   authors                165 non-null    object  
 2   year                   163 non-null    UInt16  
 3   type_of_paper          163 non-null    category
 4   journal                161 non-null    object  
 5   name_of_univerity      73 non-null     object  
 6   type_of_study          164 non-null    object  
 7   pre_prints             165 non-null    object  
 8   doi                    159 non-null    object  
 9   used                   161 non-null    category
 10  partially              71 non-null     category
 11  target_authors         73 non-null     category
 12  method                 73 non-null     category
 13  hybridisation          7 non-null      category
 14  stress_objectives      70 non-null     object  

In [9]:
# dimensions of data frame (rows and columns)
clean.shape

(165, 25)

In [10]:
clean.tail(2).T

No,170,171
publication,Enabling same-day delivery using a drone resup...,A discrete-event simulation study for pre-hosp...
authors,"Moshref-Javadi, Mohammad, Kristof P. Van Cauwe...","Wang, Zilu, and Zhaolin Hu"
year,2023,2024
type_of_paper,Journal,Journal
journal,Computational Management Science,Journal of Simulation
name_of_univerity,-,-
type_of_study,Empirical,Empirical
pre_prints,No,No
doi,https://doi.org/10.1007/s10287-023-00453-3,https://doi.org/10.1080/17477778.2024.2385994
used,Yes,Yes


In [11]:
clean.head(2).T

No,1,2
publication,How simulation modelling can help reduce the i...,The ODD protocol for describing agent-based an...
authors,"Currie, Christine SM, John W. Fowler, Kathy Ko...","Grimm, V., Railsback, S.F., Vincenot, C.E., Be..."
year,2020,2020
type_of_paper,Journal,Journal
journal,Journal of Simulation,Journal of Artificial Societies and Social Sim...
name_of_univerity,NaN,NaN
type_of_study,Discussion,Guideline
pre_prints,No,No
doi,https://www.tandfonline.com/doi/pdf/10.1080/17...,http://eprints.bournemouth.ac.uk/33918/1/7.pdf
used,No,No
